#### This Notebook contains the code used to prepare the data for analysis. It starts with the merged raw merged crime data and covariates data and performs the necessary cleaning and transformation for analysis

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import math

In [3]:
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/STAT 27420')

Mounted at /content/drive


In [ ]:
# Importing the merged crime and covariates data
did_data = pd.read_csv("offenses_merged.csv")

In [ ]:
# Following Donohue and Levitt, constructs violent crime and property crime
# variables from the smaller categories of crime
violent_crime = [
    'actual_murder','actual_manslaughter','actual_rape_total','actual_robbery_total','actual_assault_total'
]
property_crime = [
    'actual_burg_total','actual_theft_total','actual_mtr_veh_theft_total',
]

In [ ]:
did_data['violent_crime'] = did_data[violent_crime].sum(axis=1)
did_data['property_crime'] = did_data[property_crime].sum(axis=1)


In [ ]:
# Subset the raw data so only those relevant to the study remains
columns_use = [
    'agency_name',
    'state',
    'state_abb',
    'year',
    'population',
    'violent_crime',
    'property_crime',
    'employment',
    'per capita personal income',
    'poverty_rate',
    'beer_pc'

In [ ]:
# Restricting the sample to the same time window as Donohue and Levitt
# Removes any observations with negative population and employment 
subset = did_data[columns_use]
subset = subset[subset["year"]<=2014]
subset = subset[subset["year"]>=1970]
subset = subset[subset["population"]>=0]
subset = subset[subset["employment"]>=0]

In [ ]:
# Construct unique keys for each observation, 
# as some counties in different states share the same name
subset['group'] = subset['state_abb'] + subset['agency_name']
subset = subset[(subset['violent_crime']+subset['population'])>0]

In [ ]:
# only keep agencies with data available for all years in the sample
temp = subset[['year','group']].groupby(['group']).count()
temp = temp[temp['year'] == 42]
filter = list(temp.index)

In [ ]:
subset =subset[subset['group'].isin(filter)]

In [ ]:
# Create columns for crime and population the year before and after
# Used for identifying holes in the data
subset['vcrime_after'] = subset['violent_crime'].shift(1)
subset['vcrime_before'] = subset['violent_crime'].shift(-1)

subset['pcrime_after'] = subset['property_crime'].shift(1)
subset['pcrime_before'] = subset['property_crime'].shift(-1)

subset['pop_after'] = subset['population'].shift(1)
subset['pop_before'] = subset['population'].shift(-1)

In [ ]:
# For observations with one year of data missing between two years with reliable data
# fill in population or crime with the average of the year before and after
holes = (subset['vcrime_after']>30)&(subset['vcrime_before']>30)&(subset['violent_crime']==0)&(subset['year'].isin(range(1975,2013)))
subset.loc[holes,'violent_crime'] = list((subset.loc[holes,'vcrime_after']+subset.loc[holes,'vcrime_before'])*0.5)
subset.loc[holes,'property_crime'] = list((subset.loc[holes,'pcrime_after']+subset.loc[holes,'pcrime_before'])*0.5)

In [ ]:
popholes = (subset['pop_after']>1000)&(subset['pop_before']>1000)&(subset['population']==0)&(subset['year'].isin(range(1975,2013)))
subset.loc[popholes,'population'] = list((subset.loc[popholes,'pop_after']+subset.loc[popholes,'pop_before'])*0.5)

In [ ]:
subset['jobpop ratio'] = subset['employment']/subset['population']
subset=subset.drop(columns=['employment','state','vcrime_after', 'vcrime_before','pcrime_after','pcrime_before','pop_before','pop_after'])

In [ ]:
# Creating a copy to perform the transformations on
subset_logdiff = subset.copy()

In [ ]:
# Perform the log transformation on relevant data
subset_logdiff[['population',
        'violent_crime',
    'property_crime',
    'jobpop ratio',
    'per capita personal income',
    'poverty_rate',
    'beer_pc']]= subset[['population',
        'violent_crime',
    'property_crime',
    'jobpop ratio',
    'per capita personal income',
    'poverty_rate',
    'beer_pc']].apply(np.log)


In [ ]:
# Set any infinity values to 0
# Since we know they arise from natural log of 0
subset_logdiff = subset_logdiff.replace([np.inf, -np.inf], 0)

In [ ]:
# Calculating the different of the natural logs
# Data is ordered with later years at the top
# Thus I take the neative o
subset_logdiff[['population',
        'violent_crime',
    'property_crime',
    'jobpop ratio',
    'per capita personal income',
    'poverty_rate',
    'beer_pc']]= -subset_logdiff[['population',
        'violent_crime',
    'property_crime',
    'jobpop ratio',
    'per capita personal income',
    'poverty_rate',
    'beer_pc']].diff()


In [ ]:
# Remove year = 2014 because that equals crime from another state subtract this state in 2014
subset_logdiff = subset_logdiff[subset_logdiff["year"]<2014]
subset_logdiff["year"] = subset_logdiff["year"]+1

In [ ]:
# Set extreme values for logdiff violent crime, property crime, and population crimen to NaN
# Observing the data close tells me that most arise from 
# holes, i.e. one year of data missing and reported as 0, 
# in the data that are more than 1 year
# Chose not to do average or backfill because it seems unreasonable to such assumptions
subset_logdiff.loc[(abs(subset_logdiff['violent_crime'])>3),'violent_crime'] = float('NaN')
subset_logdiff.loc[(abs(subset_logdiff['property_crime'])>3),'property_crime'] = float('NaN')
subset_logdiff.loc[(abs(subset_logdiff['population'])>5),'population'] = float('NaN')

In [ ]:
subset_logdiff.to_csv('logdiff_did.csv')
files.download('logdiff_did.csv')

In [6]:
subset['early_leg'] = subset['state_abb'].isin(['WA','CA','AK','HI','NY'])

#### Preparing data for Conditional Parallel Trends in R

In [7]:
# Defining the treatment time variable
subset['lag15'] = (subset['year']>=1985)

In [23]:
# Once again subseting the data 
# since we only need certain variables in the pre-treatment
subset_clean = subset[[
    'agency_name',
    'state_abb',
    'year',
    'population',
    'violent_crime',
    'property_crime',
    'jobpop ratio',
    'per capita personal income',
    'poverty_rate',
    'beer_pc',
    'early_leg',
    'lag15',
    'group']]


In [24]:
# Difference the outcome variables once again
subset_clean[['logd_violent_crime', 'logd_property_crime']] = -subset_clean[['violent_crime', 'property_crime']].diff()

In [25]:
subset_clean = subset_clean[subset_clean['year']<1990]

In [ ]:
subset_clean = subset_clean.replace([np.inf, -np.inf], 0)
subset_clean = subset_clean.replace([np.inf, -np.inf], 0)

In [27]:
subset_clean.to_csv('conditional_parallel_trend.csv')
from google.colab import files
files.download('conditional_parallel_trend.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>